<div class="alert alert-block alert-info">
<h2 style="margin-top: 0">In this notebook, we will cover:</h2>

- An overview of astroquery
- Example use of several archives

This notebook builds on the previous tutorials for Astropy Units/Quantities, Coordinates, FITS, and Tables.
</div>

In [ ]:
# Import useful Astropy functionality
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

## Downloading a Catalog from a Paper

In [ ]:
from astroquery.vizier import Vizier
catalog_list = Vizier.find_catalogs('Dupuy Liu')
[print(f'{k}:{v.description}') for k,v in catalog_list.items()]

In [ ]:
result = Vizier.get_catalogs(catalog="J/ApJS/201/19")[0]

In [ ]:
result

In [ ]:
result[0]

Exercise: Think about a paper in your field and get the data from it.

## Cone Search SDSS

Hint: most astroquery modules have a query_region() method when it makes sense to query for coordinate.

In [ ]:
# First create a SkyCoord object
position = SkyCoord(result[0]['RAJ2000'], result[0]['DEJ2000'], unit="deg")

# Get closeby sources from SDSS
from astroquery.sdss import SDSS
sdss = SDSS()
closeby_sources = sdss.query_region(position, radius=0.1 * u.arcmin, timeout=3600)

In [ ]:
closeby_sources

In [ ]:
images = sdss.get_images(matches=closeby_sources, band =['u', 'g', 'r', 'i', 'z'])

In [ ]:
np.shape(images)

In [ ]:
for image in images:
    print(image[0].header['Filter'])

In [ ]:
images[0][0].header

In [ ]:
plt.imshow(images[0][0].data, vmin=0.1, vmax=1)

See how to make a RGB image: https://docs.astropy.org/en/stable/visualization/rgb.html

## Search MAST archive

In [ ]:
from astroquery.mast import Observations
obs_table = Observations.query_object("M8",radius=".02 deg")
len(obs_table)

In [ ]:
obs_table = Observations.query_criteria(dataproduct_type=["image"],
                                        proposal_pi="*Dokkum*",
                                        project='HST',
                                        instrument_name='WFC3/IR',
                                        obs_collection='HST',
                                        filters='F140W',
                                        proposal_id = '12177')

In [ ]:
len(obs_table)

In [ ]:
obs_table[0]['obs_id']

In [ ]:
data_products_by_obs = Observations.get_product_list(obs_table[0])
data_products = data_products_by_obs[(data_products_by_obs['obs_id'] == 'ibhj01j9q') & \
                                    (data_products_by_obs['productType'] == 'SCIENCE')]

In [ ]:
data_products

In [ ]:
Observations.download_products(data_products) 

JWST

In [ ]:
obs_table = Observations.query_criteria(proposal_id="1571",
                                        instrument_name = "NIRISS*")

In [ ]:
id = []
filters = [obs.split(';')[:] for obs in obs_table['filters']]
np.unique(filters)

In [ ]:
clear = ['CLEAR' in f for f in filters]
blue = [(f == 'GR150R;F115W' or f == 'GR150C;F115W') for f in obs_table['filters']]
green = [(f == 'GR150R;F150W' or f == 'GR150C;F150W') for f in obs_table['filters']]
red = [(f == 'GR150R;F200W' or f == 'GR150C;F200W') for f in obs_table['filters']]

In [ ]:
plt.figure(figsize=[15,15])
plt.plot(obs_table['s_ra'][clear]+0.5, obs_table['s_dec'][clear], 'o', markersize=10, mfc='black', mec='white', alpha=0.2)
plt.plot(obs_table['s_ra'][blue]-0.5, obs_table['s_dec'][blue], 'o', markersize=10, mfc='blue', mec='white',alpha=0.2)
plt.plot(obs_table['s_ra'][green], obs_table['s_dec'][green]+0.5, 'o', markersize=10, mfc='green', mec='white',alpha=0.2)
plt.plot(obs_table['s_ra'][red], obs_table['s_dec'][red]-0.5, 'o', markersize=10, mfc='red', mec='white',alpha=0.2)

In [ ]:
mask = [(ra > 155) and (ra < 165) and (dec > 30) and (dec < 40) for ra, dec in zip(obs_table['s_ra'], obs_table['s_dec'])]
new_obs_table = obs_table[mask]

In [ ]:
product_list = [Observations.get_product_list(obs) for obs in obs_table[mask]]
products = vstack(product_list)
len(products)

In [ ]:
filtered = Observations.filter_products(products,productType=['SCIENCE','PREVIEW'],
                                            extension=['_rate.fits','_rate.jpg'])
print(len(filtered))
filtered

In [ ]:
Observations.download_products(filtered, download_dir='RAW/')

## Archives with TAP Services

TAP stands for Table Access Protocol which is a standard way that many archives provide access to data. This is a Virtual Observatory (VO) standard and you can find much more about searching and accessing TAP tables in the extensive [NAVO Notebooks](https://nasa-navo.github.io/navo-workshop/index.html). Here I will show a couple of simple examples.

Some astroquery modules are simply wrappers around TAP services, for example GAIA. 

The examples below is taken from the [Astronomical Data Carpentries lesson](https://datacarpentry.org/astronomy-python/).

In [ ]:
from astroquery.gaia import Gaia

In [ ]:
tables = Gaia.load_tables(only_names=True)
tables

In [ ]:
for table in tables:
    print(table.name)

There are lots of table, here are some examples:

- gaiadr2.gaia_source, which contains Gaia data from data release 2,

- gaiadr2.panstarrs1_original_valid, which contains the photometry data we’ll use from PanSTARRS, and

- gaiadr2.panstarrs1_best_neighbour, which we’ll use to cross-match each star observed by Gaia with the same star observed by PanSTARRS.

You can probably infer what many of these tables and columns are by looking at the names, but you should resist the temptation to guess. To find out what the columns mean, [read the documentation](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/).

In [ ]:
meta = Gaia.load_table('gaiadr2.gaia_source')
meta

In [ ]:
print(meta)

In [ ]:
# Print the columns in the table
for column in meta.columns:
    print(column.name)

This is just the metadata. How do you get the actual data. You don't. The point of archives like this one and other large archives is that you do not need to download the full data but you can filter it on the remote server and only download data that satisfies your search criteria. The query is written in a language similar to SQL (structured query language) which has had some astro functionality added called ADQL (the Astronomical Data Query Language).  

Here is a simple ADQL query:

In [ ]:
query1 = """SELECT 
TOP 10
source_id, ra, dec, parallax 
FROM gaiadr2.gaia_source
"""

In [ ]:
job = Gaia.launch_job(query1)
job

In [ ]:
print(job)

In [ ]:
results = job.get_results()
type(results)

In [ ]:
results

## Astrometry.net in the Notebook Example

In order to use astrometry.net in the notebook, we need to specify a key. Instructions are on the astroquery website:

https://astroquery.readthedocs.io/en/latest/astrometry_net/astrometry_net.html

A short summary:
- Create an account on the astrometry.net website. You can log in with Google, GitHub, etc.
- Go to the My Profile section or [here](https://nova.astrometry.net/api_help) to find your API key
- Create a file for the configuration, it needs to live in a specific location: `touch ~/.astropy/config/astroquery.cfg`
- Open the file with a text editor and add the content below:
```
[astrometry_net]

## The Astrometry.net API key.
api_key = KEY FROM THE WEBSITE

## Name of server
server = http://nova.astrometry.net

## Default timeout for connecting to server
timeout = 120
```

In [ ]:
from astroquery.astrometry_net import AstrometryNet
from astropy.io import fits

fit = False

ast = AstrometryNet()
if fit == True:
    wcs_header = ast.solve_from_image('../WCS/rozhen2m_1.fits', force_image_upload=True)
    day4_header = fits.getheader('../WCS/new_rozhen2m_1.fits')
    diff = fits.diff.HeaderDiff(wcs_header, day4_header)
    print(diff.report())


[Astroquery](https://astroquery.readthedocs.io/en/latest/) is an affiliated package in the Astropy Project. Most of the functionality is developed by the archives themselves. Let's look at the list of archives which are represented.

## Resources

Notebook from Erik and Brigitta:

https://github.com/AstroHackWeek/AstroHackWeek2019/blob/master/day2_software/Querying%20with%20Astroquery.ipynb

MAST notebooks repo:

https://spacetelescope.github.io/mast_notebooks/intro.html

CDS:

https://github.com/cds-astro/tutorials

GAIA:

https://allendowney.github.io/AstronomicalData/01_query.html

ALMA:

https://almascience.eso.org/alma-data/archive/archive-notebooks

ESA:

https://www.cosmos.esa.int/web/hst/astroquery

NAVO workshops:

https://nasa-navo.github.io/navo-workshop/index.html

NOIRLab DataLab

https://datalab.noirlab.edu/

PyESASky: 

https://github.com/esdc-esac-esa-int/pyesasky/tree/master